In [ ]:
# https://github.com/StatBiomed/UniTVelo
# conda create -n unitvelo python=3.7
# conda activate unitvelo
import unitvelo as utv
import scvelo as scv
import numpy as np
import pandas
import importlib_metadata
scv.set_figure_params(dpi=200, fontsize=16, facecolor='none')
# Arguments:
# -- velo.R2_ADJUST (bool), linear regression R-squared on extreme quantile (default) or full data (adjusted)
# -- velo.IROOT (str), specify root cell cluster would enable diffusion map based time initialization, default None
# -- velo.FIT_OPTION (str), '1' Unified-time mode (default), '2' Independent mode
# -- velo.GPU (int), specify the GPU card used for fitting, -1 will switch to CPU mode, default 0.

velo_config = utv.config.Configuration()
# velo_config.R2_ADJUST = True 
# velo_config.IROOT = None
# velo_config.FIT_OPTION = '1'
# velo_config.GPU = '-1'
# velo_config.AGENES_R2 = 1

# from Fig3 of Unitvelo paper, now with modifications
velo_config.MIN_SHARED_COUNTS = 20 # def 20
velo_config.N_TOP_GENES = 2000 # def 2000
velo_config.N_PCS = 50  # def 30
# https://github.com/theislab/scvelo/issues/112  Could you check how the stoch. model behaves as you decrease the number of neighbors in scv.pp.moments(adata, n_neighbors=10). I think 30 neighbors on 100 cells for second-oder moments is way too much. That default value was selected when testing against datasets of 3k - 35k cells
velo_config.N_NEIGHBORS = 30 # def 30
velo_config.VGENES = 'offset'
# done velo_config.VGENES = 'offset'
velo_config.R2_ADJUST = False
velo_config.IROOT = 'Tpex1' # def None 'Tpex'
velo_config.FIT_OPTION = '1'
# velo_config.NUM_REP = 2
        # (int) number of random initializations of time points, default 1
        # in rare cases, velocity field generated might be reversed, possibly because stably and monotonically changed genes
        # change this parameter to 2 might do the trick
        # self.NUM_REP = 1
        # when self.NUM_REP = 2, the following parameter will determine how the second time will be initialized 



# (str) selection creteria for velocity genes used in RNA velocity construction, default basic
        # 1. raws, all highly variable genes specified by self.N_TOP_GENES will be used
        # 2. offset, linear regression $R^2$ and coefficient with offset, will override self.R2_ADJUST
        # 3. basic, linear regression $R^2$ and coefficient without offset
        # 4. single gene name, fit this designated gene alone, for model validation purpose only
        # 5. [list of gene names], manually provide a list of genes as velocity genes in string, might improve performance, see scNT
        # self.VGENES = 'basic'




label = 'clusters'
exp_metrics = {}

In [ ]:
# # generated using velocyto to extract splicing info

adata_OTW1 = scv.read("starsolo_sorted_renamedCB_OTW1_BKUJN.loom", cache=True)
adata_OTW2 = scv.read("starsolo_sorted_renamedCB_OTW2_WZZQC.loom", cache=True)
scv.utils.clean_obs_names(adata_OTW1)
scv.utils.clean_obs_names(adata_OTW2)


adata_OTK1 = scv.read("starsolo_sorted_renamedCB_OTK1_XK8RL.loom", cache=True)
adata_OTK2 = scv.read("starsolo_sorted_renamedCB_OTK2_GYNSM.loom", cache=True)

scv.utils.clean_obs_names(adata_OTK1)
scv.utils.clean_obs_names(adata_OTK2)



# generated from seurat object from H5AD
# adata_seurat_h5ad = scv.read('05_reclustered_ovarT_integrated_noRefUsed_seurat_obj.h5ad', cache=True) 
# scv.utils.clean_obs_names(adata_seurat)

# # generated from seurat object from LOOM
adata_seurat= scv.read('03_reclustered_ovarT_integrated_noRefUsed_seurat_obj_LOOM.loom', cache=True) 


# adata_seurat= scv.read('04_reclustered_ovarT_integrated_noRefUsed_seurat_obj_LOOM.loom', cache=True) 



In [ ]:
adata_seurat

In [ ]:
adata_seurat.obs['condition']

In [ ]:
# filter only cells from individual samples
adata_seurat_OTW1 = adata_seurat[adata_seurat.obs['sample'].isin(['OTW1'])].copy()                       
adata_seurat_OTW1.obs['condition']
scv.utils.clean_obs_names(adata_seurat_OTW1)

adata_seurat_OTW2 = adata_seurat[adata_seurat.obs['sample'].isin(['OTW2'])].copy()                       
adata_seurat_OTW2.obs['condition']
scv.utils.clean_obs_names(adata_seurat_OTW2)

adata_seurat_OTK1 = adata_seurat[adata_seurat.obs['sample'].isin(['OTK1'])].copy()                       
adata_seurat_OTK1.obs['condition']
scv.utils.clean_obs_names(adata_seurat_OTK1)

adata_seurat_OTK2 = adata_seurat[adata_seurat.obs['sample'].isin(['OTK2'])].copy()                       
adata_seurat_OTK2.obs['condition']
scv.utils.clean_obs_names(adata_seurat_OTK2)
adata_seurat_OTK2
# # filter only WT cells
# adata_seurat_wt = adata_seurat[adata_seurat.obs['condition'].isin(['WT'])].copy()                       
# adata_seurat_wt.obs['condition']

In [ ]:
# # combine replicates together with seurat, this function only works with 2 datasets so I need to split the command and then not merge but concatenate the files
# adata1 = scv.utils.merge(adata_OTW1, adata_seurat_OTW1)
# adata2 = scv.utils.merge(adata_OTW2, adata_seurat_OTW2)
# adata3 = scv.utils.merge(adata_OTK1, adata_seurat_OTK1)
# adata4 = scv.utils.merge(adata_OTK2, adata_seurat_OTK2)


adata1 = scv.utils.merge(adata_seurat_OTW1, adata_OTW1)
adata2 = scv.utils.merge(adata_seurat_OTW2, adata_OTW2)
adata3 = scv.utils.merge(adata_seurat_OTK1, adata_OTK1)
adata4 = scv.utils.merge(adata_seurat_OTK2, adata_OTK2)


adata_wt = adata1.concatenate(adata2)
adata_ko = adata3.concatenate(adata4)
adata_ko

In [ ]:
# import existing data
# adata.write('adata_scvelo.h5ad', compression='gzip')
# adata.write('adata_scvelo.loom')

# adata = scv.read('adata_scvelo.h5ad')
# adata
adata_wt

In [ ]:
# import UMAP coordinates from seurat metadata
adata_wt.obsm['X_tsne'] = np.stack((adata_wt.obs['tSNE_1'], adata_wt.obs['tSNE_2']), axis=-1)
adata_wt.obs['clusters'] = adata_wt.obs['reclustering']

adata_ko.obsm['X_tsne'] = np.stack((adata_ko.obs['tSNE_1'], adata_ko.obs['tSNE_2']), axis=-1)
adata_ko.obs['clusters'] = adata_ko.obs['reclustering']

In [ ]:
# Visualize proportions of spliced and unspliced transcripts

scv.pl.proportions(adata_wt, save='wt.svg')

# scv.pl.proportions(adata_wt)


In [ ]:
# Run model (label refers to column name in adata.obs specifying celltypes)

# sometimes this gets stuck so you have to remove or rename the folder in res/temp and then simply rerun this block and it should work
import importlib_metadata
adataM_wt = utv.run_model(adata_wt, label, config_file=velo_config)

In [ ]:
# I did not run these prior to the model
# scv.pp.filter_and_normalize(adata_wt, min_shared_counts=20, n_top_genes=2000, subset_highly_variable=False) #using this filtering on this single small replicate, I end up with only 6 genes with the default filtering; default 20 and 2000
# # scv.pp.moments(adata, n_pcs=30, n_neighbors=10) # default is 30 for neighbors, but they recommend to try lower numbers like 10 when dealing with low number of total cells

# scv.tl.pca(adata_wt)
# scv.pp.neighbors(adata_wt, n_pcs=30, n_neighbors=30, random_state=0)
# scv.pp.moments(adata_wt, n_pcs=None, n_neighbors=None)

In [ ]:
# Visualize proportions of spliced and unspliced transcripts
scv.pl.proportions(adata_ko, save='ko.svg')

# scv.pl.proportions(adata_ko)

In [ ]:
# I did not run these prior to the model
# scv.pp.filter_and_normalize(adata_ko, min_shared_counts=20, n_top_genes=2000, subset_highly_variable=False) #using this filtering on this single small replicate, I end up with only 6 genes with the default filtering; default 20 and 2000
# # scv.pp.moments(adata, n_pcs=30, n_neighbors=10) # default is 30 for neighbors, but they recommend to try lower numbers like 10 when dealing with low number of total cells

# scv.tl.pca(adata_ko)
# scv.pp.neighbors(adata_ko, n_pcs=30, n_neighbors=30, random_state=0)
# scv.pp.moments(adata_ko, n_pcs=None, n_neighbors=None)

In [ ]:
# Run model (label refers to column name in adata.obs specifying celltypes)
import importlib_metadata
adataM_ko = utv.run_model(adata_ko, label, config_file=velo_config)

In [ ]:
# # save if needed but normally it is automatically saved from the model
# adataM_wt.write('./res/wt_tsne_120424_recluster2/temp_1.h5ad', compression='gzip')

adataM_ko.write('./res/ko_tsne_120424_recluster2/temp_1.h5ad', compression='gzip')

In [ ]:
adataM_ko

In [ ]:
adataM_wt

In [ ]:
# saved temporary data from the model
import os
os.listdir('./res/ko_tsne_120424_recluster2')

In [ ]:
adataM = scv.read('./res/wt_tsne_101824_recluster2/temp_1.h5ad')
adataM_wt=adataM

# change colors if needed

adataM = scv.read('./res/ko_tsne_101824_recluster2/temp_1.h5ad')
# adataM = adataM_ko
adataM_ko=adataM

In [ ]:
# see present colors
adataM.uns['clusters_colors']

# array(['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2'], dtype=object) ...for KO
# ['Tdual', 'Teff', 'Tex', 'Tpex', 'Tprol1', 'Tprol2', 'Ttex']
# ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b'] ..for WT
# ['Teff', 'Tex', 'Tpex', 'Tprol1', 'Tprol2', 'Ttex']

# # original
# adataM.uns[f'{label}_colors'] = ['#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b','#e377c2']






In [ ]:
# # import UMAP coordinates from seurat metadata
#adataM.obsm['X_tsne'] = np.stack((adataM.obs['tSNE_1'][::-1], adataM.obs['tSNE_2'][::-1]), axis=1) # this reverses the order but doesn't fix the problem
# adataM.obsm['X_tsne'] = np.stack((adataM.obs['tSNE_1'], adataM.obs['tSNE_2']), axis=-1)
# adataM.obsm['X_umap'] = np.stack((adataM.obs['UMAP_1'], adataM.obs['UMAP_2']), axis=-1)
scv.pl.velocity_embedding_stream(adataM, color=label, dpi=120, title='Re-clustered CD8 OT-I',basis='tsne')
# adata_wt.obs['clusters'] = adata_wt.obs['reclustering']

# adata_ko.obsm['X_tsne'] = np.stack((adata_ko.obs['tSNE_1'], adata_ko.obs['tSNE_2']), axis=-1)
# adata_ko.obs['clusters'] = adata_ko.obs['reclustering']

In [ ]:
## THIS GIVES THE MOST IMPORTANT PLOT FOR PUBLICATION ...and repeat it also for KO
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
adataM = adataM_wt
# for WT only change the colors so that they correspond to KO
adataM.uns[f'{label}_colors'] = ["#00BFC4","#B79F00","#00BA38","#F8766D","#619CFF","#F564E3"]


scv.set_figure_params(style='scvelo', dpi=150, dpi_save=400, frameon=None, vector_friendly=True, transparent=True, fontsize=12, figsize=[4,5])
scv.pl.velocity_embedding(adataM, arrow_length=3, arrow_size=2, basis='tsne', save='embedding_wt.svg')
scv.pl.velocity_embedding_grid(adataM, arrow_length=3, arrow_size=2, basis='tsne', save='embedding_grid_wt.svg')
scv.pl.velocity_embedding_stream(adataM, color=label, title='Re-clustered CD8 OT-I',basis='tsne', save='embedding_stream_wt.svg')

# adataM

In [ ]:
adataM

In [ ]:
# Parameter `cluster_edges` is for algorithm evaluation purpose given expert annotated ground truth. It contains a list of tuples in which stores the source cluster and target cluster of cells.
# cluster_edges = [
#     ("Tpex"), 
#     ("Tprol1", "Tprol2"),
#     ("Teff", "Tdual"), 
#     ("Tex"),("Ttex")]

In [ ]:
scv.tl.latent_time(adataM, min_likelihood=None)
scv.pl.scatter(adataM, color='latent_time', color_map='gnuplot', size=20, dpi=400,basis='tsne')

In [ ]:
subvar = adataM.var.loc[adataM.var['velocity_genes'] == True]
sub = adataM[:, subvar.index]

In [ ]:
# save all the genes with latent times
sub.var.to_csv('genes_all_variable_latent_time_KOonly.txt', sep='\t', index=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.displot(sub.var['fit_t'].values, kde=True, bins=20)
plt.xticks([0, 0.5, 1], fontsize=13)
plt.yticks(fontsize=13)
plt.ylabel('Number of Genes', fontsize=15)
plt.title('Peak Time', fontsize=15)

In [ ]:
scv.tl.rank_velocity_genes(adataM, groupby='clusters', min_corr=.3)
df = pandas.DataFrame(adataM.uns['rank_velocity_genes']['names'])
df.head(n=100)

In [ ]:
kwargs = dict(frameon=False, size=10, linewidth=1.5,
              add_outline='Tpex1,Ttex1')

scv.pl.scatter(adataM, df['Tpex1'][:5], ylabel='Tpex1', **kwargs)
scv.pl.scatter(adataM, df['Ttex1'][:5], ylabel='Ttex1', **kwargs)

In [ ]:
scv.pl.scatter(adataM, 'Bcl11b', color=['clusters', 'velocity'])

In [ ]:
# adataM_wt =adataM
# adataM_ko=adataM
adataM=adataM_wt
scv.pl.velocity(adataM, ['Il7r','Bcl2', 'Tbx21','Ikzf1','Pdcd1','Havcr2','Ccl5','Gzma','Bcl11b'], ncols=2,basis='tsne')
# Il7r Bcl2 Tbx21Ikzf1 Pdcd1 HAVCR Ccl5 Gzma 
# # scv.pl.velocity(adataM, ['Il7r','Tcf7','Bach2','Slamf6','Cd55', 'Bcl2', 'Tbx21','Ikzf1','Pdcd1','Ctla4','Havcr2','Tox','Ccl1','Ccl5','Gzma','Gzmk'], ncols=2,basis='tsne')

#  'Cd44',  'Gzma', ,'Tcf7','Tox'

In [ ]:
# repression genes
gene = sub.var.loc[sub.var['fit_t'] < 0].index # repression gene
with open("genes_KOonly_repression_raw.txt", "w") as outfile:
    outfile.write("\n".join(gene))

# genes_repression = open('genes_repression.txt', 'r')

In [ ]:
scv.tl.score_genes_cell_cycle(adataM)
scv.pl.scatter(adataM, color_gradients=['S_score', 'G2M_score'], smooth=True, perc=[5, 95], dpi=120,basis='tsne')

In [ ]:
scv.tl.velocity_confidence(adataM)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(adataM, c=keys, cmap='coolwarm', perc=[5, 95], dpi=120,basis='tsne')

In [ ]:
df = adataM.obs.groupby('clusters')[keys].mean().T
df.style.background_gradient(cmap='coolwarm', axis=1)

In [ ]:
scv.tl.terminal_states(adataM)
scv.pl.scatter(adataM, color=["root_cells", "end_points"],basis='tsne')

In [ ]:
scv.pl.velocity_graph(adataM, threshold=.8,basis='tsne')

In [ ]:
x, y = scv.utils.get_cell_transitions(adataM, basis='tsne', starting_cell=100)
ax = scv.pl.velocity_graph(adataM, c='lightgrey', edge_width=.05, show=False, dpi=120, basis='tsne')
ax = scv.pl.scatter(adataM, x=x, y=y, s=120, c='ascending', cmap='gnuplot', ax=ax, dpi=120, basis='tsne')

In [ ]:
scv.tl.velocity_pseudotime(adataM)
scv.pl.scatter(adataM, color='velocity_pseudotime', cmap='gnuplot', basis='tsne')

In [ ]:
# this is needed due to a current bug - bugfix is coming soon.
adataM.uns['neighbors']['distances'] = adataM.obsp['distances']
adataM.uns['neighbors']['connectivities'] = adataM.obsp['connectivities']

scv.tl.paga(adataM, groups='clusters')
df = scv.get_df(adataM, 'paga/transitions_confidence', precision=2).T
df.style.background_gradient(cmap='Blues').format('{:.2g}')

In [ ]:
scv.pl.paga(adataM, basis='tsne', size=50, alpha=.1,
            min_edge_width=2, node_size_scale=1.5)

In [ ]:
# plot the heatmap for repression genes
# change genes that we don't want to show in the heatmap to None and then provide it for: yticklabels=['gene 1', None, None, None, 'gene 5'] https://github.com/theislab/scvelo/issues/445

scv.pl.heatmap(
    adataM, var_names=gene, sortby='latent_time', yticklabels=['Smap1',None,None,'Creb1',None,None,None,None,None,None,'Kif14',None,None,None,None,None,None,None,None,'Fyn','Prf1',None,None,None,None,None,None,None,None,None,None,None,None,'Ikzf1',None,None,None,None,None,None,'Havcr2',None,'Ncor1',None,None,None,None,None,None,None,None,None,None,None,None,'Stat3',None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,'Gzma','Gzmk',None,None,None,None,None,None,None,None,None,None,'Il7r',None,None,None,None,'Ep300',None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,'Foxp4',None,'Satb1',None,None,None,None,'Brd8',None,None,None,None,None,None,None,None,None,None,None,None,'Gata3',None,'Il2ra',None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,'S1pr1',None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,'Runx3',None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,'Mki67',None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None],
    col_color=label, n_convolve=100)
    # save='heatmap_genes_repression.png')




In [ ]:
adataM
top_genes = adataM.var['fit_t'].sort_values(ascending=False).index
# top_genes = adataM.var['fit_likelihood'].sort_values(ascending=False).index[:300]
scv.pl.heatmap(adataM, var_names=top_genes, sortby='latent_time', col_color='clusters', n_convolve=100)

In [ ]:
top_genes


In [ ]:
# induction genes
gene = sub.var.loc[sub.var['fit_t'] > 0.95].index
with open("genes_KOonly_induction_raw.txt", "w") as outfile:
    outfile.write("\n".join(gene))

# # save the induction genes
# genes_repression = open('genes_induction.txt', 'r')

In [ ]:
# change genes that we don't want to show in the heatmap to None and then provide it for: yticklabels=['gene 1', None, None, None, 'gene 5'] https://github.com/theislab/scvelo/issues/445
scv.pl.heatmap(
    adataM, var_names=gene, sortby='latent_time', yticklabels=[None,None,None,None,None,None,None,None,None,'Tcf7',None,None,None,None,None,'Ccl1','Ccl5',None,None,'Nfkbia',None,None,None,None,None,None,None,None,None,None,None,'Sub1',None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,'Nr4a2',None,None,None,None,None,None,None,None,'Nr4a3',None,None,None,None,'Cd52',None,None,None,None,None,None,None,None,None,None,None,None,None,'Klrg1','Klrb1a','Klre1','Klrd1','Klrh1','Klra5',None,None,'Tyrobp',None,None,None,None,'Cd37',None,None,None,None,None,None,None,None,None,None,None,'Cd3g',None,None,'Ccr2',None,None],
    col_color=label, n_convolve=100)
    # save='heatmap_genes_induction.png')



In [ ]:
utv.pl.plot_range('Cd44', adataM, velo, show_legend=True, show_ax=True)

In [ ]:
# CONTINUE BY FIGURING OUT HOW TO GET PROPER TABLE OF REPRESSION and induction genes, export them and label appropriate genes to the heatmap

In [ ]:
subvar = adataM.var.loc[adataM.var['velocity_genes'] == True]
sub = adataM[:, subvar.index]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.displot(sub.var['fit_t'].values, kde=True, bins=20)
plt.xticks([0, 1], fontsize=13)
plt.yticks(fontsize=13)
plt.ylabel('Number of Genes', fontsize=15)
plt.title('Peak Time', fontsize=15)

In [ ]:
r2 = sub.var[['fit_t', 'fit_sr2', 'fit_ur2']].sort_values(by=['fit_sr2'], ascending=False)
r2

In [ ]:
utv.pl.plot_range('Tcf7', adataM, velo_config,
    show_legend=False, show_ax=False)

In [ ]:
gene_name = 'Tcf7'
adataM.obs['temp'] = adataM[:, gene_name].layers['Ms']
scv.pl.scatter(adataM, color='temp', color_map='viridis', size=20, title='')

In [ ]:
# copy this WTonly and make it separately for WT and KO, then produce the files with 30 neighbors, root Tpex, offset